# Predict the weather 📈🌦 For Airport with Prophet

💬 (Boss): *alpha苏泽, I need a weather forecast for New York state*. 

&nbsp;&nbsp;&nbsp;↪️ (Me): *What aspect of the weather would you like to know about?*

&nbsp;&nbsp;&nbsp;↪️ (Boss): *Temperature and wind metrics would be great!*

💬 (Me): *What will this analysis be used for?*

&nbsp;&nbsp;&nbsp;↪️ (Boss): *We'd like to sell this as a product to Airports.*

💬 (Me): A minute please 😎.

---


<blockquote style='
    font-size: 16px;
    color: #171716;
    border-left: 4px solid #3793EF;
    padding-left: 16px;
    line-height: 1.6;
    margin: -10px 0 0 0px;
'>
The data source: <a href='https://cloud.google.com/bigquery/public-data#sample_tables'>google cloud weather dataset</a> from Google Big Query. 
</blockquote>

In [18]:
import altair as alt
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly

In [19]:
def clean_data(df):
    df['date']=pd.to_datetime(df.date)
    df['max_gust']=df['max_gust'].astype(float).replace(999.9, np.nan) 
    df['max_temp']=df['max_temp'].astype(float).replace(9999.9, np.nan) 
    df['min_temp']=df['min_temp'].astype(float).replace(9999.9, np.nan) 
    return df

## Weather data

In [20]:
weather = pd.read_csv("weather-station.csv")
weather

,date,name,min_temp,max_temp,max_gust
0,2020-08-18,EAST HAMPTON AIRPORT,68.1,69.8,15.0
1,2020-08-17,EAST HAMPTON AIRPORT,67.7,75.9,18.1
2,2020-08-16,EAST HAMPTON AIRPORT,67.5,68.9,15.0
3,2020-08-15,EAST HAMPTON AIRPORT,71.2,74.3,NaN
4,2020-08-14,EAST HAMPTON AIRPORT,75.5,84.2,18.1
...,...,...,...,...,...
1670,2016-01-09,EAST HAMPTON AIRPORT,43.3,48.2,15.0
1671,2016-01-08,EAST HAMPTON AIRPORT,34.2,46.4,20.0
1672,2016-01-07,EAST HAMPTON AIRPORT,38.5,45.5,NaN
1673,2016-01-06,EAST HAMPTON AIRPORT,36.3,41.0,NaN


In [21]:
weather=clean_data(weather)

## Predict weather data

Select the weather feature to predict

In [22]:
feature = 'min_temp'

In [23]:
weather_subset=weather[['date', feature]]
weather_subset.columns=['ds', 'y']

m = Prophet(daily_seasonality=True)
m.fit(weather_subset)
future = m.make_future_dataframe(periods=365)
forecast = m.predict(future)

INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


## Plot weather forecast

In [24]:
my_plot=plot_plotly(m, forecast)

my_plot

In [25]:
my_comps=plot_components_plotly(m, forecast)
my_comps.layout.update({'title': 'Trends grouped by time units (e.g., daily, yearly, weekly'})
my_comps